In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='7um'

In [ ]:
design.get_chip_z('Q_chip')

'7um'

In [26]:
q0

[name:    Q0
 class:   TransmonPocket6       
 options: 
   'pos_x'             : '-300um',                     
   'pos_y'             : '-500um',                     
   'orientation'       : '0.0',                        
   'chip'              : 'Q_chip',                     
   'layer'             : '1',                          
   'connection_pads'   : {
                         },
   'pad_gap'           : '30um',                       
   'inductor_width'    : '20um',                       
   'pad_width'         : '455um',                      
   'pad_height'        : '90um',                       
   'pocket_width'      : '650um',                      
   'pocket_height'     : '650um',                      
   'hfss_wire_bonds'   : False,                        
   'q3d_wire_bonds'    : False,                        
   'hfss_inductance'   : '10nH',                       
   'hfss_capacitance'  : 0,                            
   'hfss_resistance'   : 0,                     

In [48]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip',
                                            pad_width='400um',
                                            pad_height='70um',
                                            pad_gap='70um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+65}um",
                                              pad_width="110um",
                                              pad_height="65um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [5]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [8]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length
4.32mm

In [11]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [12]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 +0.5 # subtract coupler length, for +0.5Ghz
resonator_length_4.append(a)
print(resonator_length_4)


[4.307562247348839]


In [24]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','800um']
jogs1[3]=['R','200um']
jogs1[4]=['R','800um']
jogs1[5]=['L','200um']
jogs1[6]=['L','800um']


In [55]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '986um', '100um')
            

# EPR Analysis

In [56]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [57]:
hfss.start()

INFO 03:33PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:33PM [load_ansys_project]: 	Opened Ansys App
INFO 03:33PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 03:33PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   KPS
INFO 03:33PM [connect_design]: 	Opened active design
	Design:    bump [Solution type: Eigenmode]
INFO 03:33PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 03:33PM [connect]: 	Connected to project "KPS" and design "bump" 😀 



True

In [58]:
hfss.activate_ansys_design("flip7um_V2", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

03:33PM 57s WARNING [activate_ansys_design]: The design_name=flip7um_V2 was not in active project.  Designs in active project are: 
['Planar', 'Design_q3d', 'flip7um_PlanarDesign', 'bump'].  A new design will be added to the project.  
INFO 03:33PM [connect_design]: 	Opened active design
	Design:    flip7um_V2 [Solution type: Eigenmode]
WARNING 03:33PM [connect_setup]: 	No design setup detected.
WARNING 03:33PM [connect_setup]: 	Creating eigenmode default setup.
INFO 03:33PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [59]:
hfss.close()


True

# LOM Analysis

In [60]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [61]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.47',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.47',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [63]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 03:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpqak62hm_.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5470000000, Maxwell, 1, False
INFO 03:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmplg6errfb.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5470000000, Maxwell, 1, False
INFO 03:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpdyiw6ar5.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5470000000, Maxwell, 2, False
INFO 03:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp3k9qcm7d.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5470000000, Maxwell, 3, False
INFO 03:59PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpi8ek3jio.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5470000000, Maxwell, 4, False
INFO 03:59

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,316.800,-99.339000,-88.533,-65.613000
pad_bot_Q0,-99.339,112.520000,-12.136,-0.090447
pad_top_Q0,-88.533,-12.136000,112.840,-11.184000
quad_coupler_c0,-65.613,-0.090447,-11.184,77.019000


In [64]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.94
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.681178 [GHz]
EC 303.026723 [MHz]
EJ 14.854145 [GHz]
alpha -348.649858 [MHz]
dispersion 14.135816 [KHz]
Lq 10.995564 [nH]
Cq 63.922508 [fF]
T1 61.094014 [us]

**Coupling Properties**

tCqbus1 -5.538637 [fF]
gbus1_in_MHz -85.669612 [MHz]
χ_bus1 -2.562137 [MHz]
1/T1bus1 2605.082427 [Hz]
T1bus1 61.094014 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.144205,358.208829,14.854145,-418.794551,65.315399,[-143.89503260943877],[-18.04753888434686],18.047539,143.895033
2,5.833276,320.571594,14.854145,-370.735691,24.016541,[-93.01121773532144],[-3.9634156139397456],3.963416,93.011218
3,5.635121,297.824559,14.854145,-342.139601,11.97079,[-87.84337812056368],[-2.4911967244340767],2.491197,87.843378
4,5.634461,297.750402,14.854145,-342.046924,11.942074,[-88.60894799666488],[-2.5320021654722633],2.532002,88.608948
5,5.683596,303.301214,14.854145,-348.993856,14.258633,[-86.43000735566619],[-2.6186749439593595],2.618675,86.430007
6,5.688004,303.802058,14.854145,-349.62165,14.485039,[-85.90698974567495],[-2.606777752110759],2.606778,85.906990
7,5.681178,303.026723,14.854145,-348.649858,14.135816,[-85.66961223015257],[-2.562136560834531],2.562137,85.669612


In [66]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
